# BB84 — Eve Role

This notebook creates the `eve_notebook.py` server script and shows how to run it.

**Pre-requisites**:

- Python 3.9+
- Install: `pip install qiskit qiskit-ibm-runtime flask requests numpy cryptography`

Make sure QiskitRuntimeService credentials are saved on machines that will call IBM backends.

In [ ]:
# Write the role script to disk
script = r"""
# eve.py
from flask import Flask, request, jsonify
import numpy as np, time, requests
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler

app = Flask(__name__)
BOB_HOST = "http://<BOB_IP>:192.168.0.107"

service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.backend("ibm_fez")
sampler = Sampler(backend=backend)

INTERCEPT_PROB = 0.25
np.random.seed(int(time.time())%100000)

def alice_prep_to_circuit(bit, basis):
    qc = QuantumCircuit(1,1)
    if bit == 1:
        qc.x(0)
    if basis == 1:
        qc.h(0)
    return qc

def measure_on_backend(prep_bit, prep_basis, meas_basis):
    qc = alice_prep_to_circuit(prep_bit, prep_basis)
    if meas_basis == 1:
        qc.h(0)
    qc.measure(0,0)
    job = sampler.run([qc], shots=1)
    result = job.result()
    try:
        r0 = result.results[0].data.counts
        bit = int(list(r0.keys())[0])
    except Exception:
        try:
            counts = result.get_counts(0)
            bit = int(list(counts.keys())[0])
        except Exception as e:
            print("Parse error:", e)
            bit = np.random.randint(2)
    return int(bit)

@app.route("/receive_from_alice", methods=["POST"])
def receive_from_alice():
    data = request.json
    preps = data.get("preps", [])
    resend_list = []
    ping_list = []
    for item in preps:
        idx = item["idx"]
        a_bit = int(item["bit"])
        a_basis = int(item["basis"])
        do_intercept = (np.random.rand() < INTERCEPT_PROB)
        if do_intercept:
            eve_basis = np.random.randint(2)
            eve_result = measure_on_backend(a_bit, a_basis, eve_basis)
            resend_list.append({"idx": idx, "bit": int(eve_result), "basis": int(eve_basis), "intercepted": True})
            if eve_basis != a_basis:
                ping_list.append(idx)
        else:
            resend_list.append({"idx": idx, "bit": a_bit, "basis": a_basis, "intercepted": False})
    r = requests.post(f"{BOB_HOST}/receive_from_eve", json={"preps": resend_list})
    return jsonify({"status":"relayed_to_bob", "bob_response": r.text, "pings": ping_list})

if __name__ == "__main__":
    print("Eve server running on port 5001")
    app.run(host="0.0.0.0", port=5001)

"""
path = '/mnt/data/eve_notebook.py'
with open(path, 'w', encoding='utf-8') as f:
    f.write(script)
print('Wrote', path)

print('\nEdit the host addresses inside the script file, then run in a terminal:')
print('python', path)


Edit BOB_HOST; ensure QiskitRuntimeService credentials are saved here.